In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

import sys
sys.path.append('../src/cnn')

from models import ConvNet
from preprocessing import LabelEncoder


physical_devices = tf.config.list_physical_devices('GPU')
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass


def read_data(path, **kwargs):
    from scipy.interpolate import interp1d
    df = pd.read_csv(path, **kwargs)
    
    # User list comprehension to create a list of lists from Dataframe rows
    data_list_of_rows = [list(row) for row in df.values]
    
    num = int(input("input a value between 0 to 3 to choose one signal amongst the 3 signals: "))
    y_axis = data_list_of_rows[num]
    
    print("You have chosen signal:", num);
    
    column_in = len(y_axis)
    col = range(0,column_in)
    x_axis = list(col)
    
    elon_list = [11, 21, 19, 18, 29]
    data_x = np.asarray(x_axis)
    data_y = np.asarray(y_axis)

    #return data_x, data_y


    # Obtain interpolation function (input original x (time) and y (signal))
    f = interp1d(data_x, data_y)
    # Create new x (same x_min and x_max but different number of data points)
    x_new = np.linspace(data_x.min(), data_x.max(), 8192)
    # Obtain new y (based on new x)
    y_new = f(x_new)
    # return both new x and new y
    return x_new, y_new


In [ ]:
NUM_TRAIN_EXAMPLES = int(1e6)
NUM_TEST_EXAMPLES = int(1e4)

NUM_CLASSES = 3
NUM_WINDOWS = 256
INPUT_SIZE = 8192

# Define label encoder to decode predictions later on
label_encoder = LabelEncoder(NUM_WINDOWS)

# Build model (should have the same parameters as the model
# in 01_cnn-train.ipynb)
model = ConvNet(
    filters=[64, 128, 128, 256, 256],
    kernel_sizes=[9, 9, 9, 9, 9],
    dropout=0.0,
    pool_type='max',
    pool_sizes=[2, 2, 2, 2, 2],
    conv_block_size=1,
    input_shape=(INPUT_SIZE, 1),
    output_shape=(NUM_WINDOWS, NUM_CLASSES),
    residual=False
)

# Load pretrained weights
# If retrained (via 01_train-cnn.ipynb), weights should be loaded as 
# outputs/weights/weight_{epoch:03d}.h5 (e.g. outputs/weights/weight_009.h5)
model.load_weights('output/weights/cnn_weights.h5')

In [3]:
# Read data (applies linear interpolation to get the right dimension)
# t = time; s = signal
t, s = read_data('../input/training.csv',header=None, sep=',')

# Normalize and add batch dimension (model expect batched data)
s_norm = s[None, :, None] / s.max()


# Pass to model and make predictions
preds = model(s_norm)[0]

# .decode will filter out predictions below threshold
# and compute the appropriate locations of the peaks
probs, locs, areas = label_encoder.decode(preds, threshold=0.5)

print("Predicted locations:\n", locs * t.max())
print("\nPredicted areas:\n", areas)
print("\nPredicted probabilities:\n", probs)

In [ ]:
# Visualize locations in the chromatogram
fig, ax = plt.subplots(1, 1, figsize=(14, 6))

ax.plot(t, s)

for (prob, loc, area) in zip(probs, locs, areas):
    y = s.min() - s.max() * 0.05 # location of the triangles
    ax.scatter(loc*t.max(), y, color='C1', marker='^', s=100, edgecolors='black')
    
ax.tick_params(axis='both', labelsize=16)
ax.set_ylabel('Signal', fontsize=16)
ax.set_xlabel('Time', fontsize=16)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
# ax.set_xlim(0.0, 3.0)